# Question 1
How many people have the job title IT staff.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sql

There are 11 tables in the chinook sample database.  The description and the database schema are below.

* ```employees``` table stores employees data such as employee id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
* ```customers``` table stores customers data.
*  ```invoices``` & ```invoice_items``` tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
* ```artists``` table stores artists data. It is a simple table that contains only the artist id and name.
* ```albums``` table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
* ```media_types``` table stores media types such as MPEG audio and AAC audio files.
* ```genres``` table stores music types such as rock, jazz, metal, etc.
* ```tracks``` table stores the data of songs. Each track belongs to one album.
* ```playlists``` & ```playlist_track``` tables: ```playlists``` table store data about ```playlists```. Each ```playlist``` contains a list of tracks. Each track may belong to multiple playlists. The relationship between the ```playlists``` table and ```tracks``` table is many-to-many. The playlist_track table is used to reflect this relationship.


![Sample Database](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

In [ ]:
try:

    # Making a connection between sqlite3
    # database and Python Program
    conn = sql.connect('chinook.db')

    # If sqlite3 makes a connection with python
    # program then it will print "Connected to SQLite"
    # Otherwise it will show errors
    print("Connected to SQLite")

    # Getting all tables from sqlite_master
    sql_query = """SELECT name FROM sqlite_master
    WHERE type='table';"""

except sqlite3.Error as error:
    print("Failed to execute the above query", error)

Connected to SQLite


# Question 1
How many people have the job title IT staff.

In [ ]:
sql_statement = """SELECT COUNT(*)
                   FROM employees
                   WHERE Title = 'IT Staff'"""

results = pd.read_sql_query(sql_statement, conn)
results

,COUNT(*)
0,2


# Question 2
Find the total number of customers per country in the database and show the results in alphabetical order by country.

In [ ]:
sql_statement = """SELECT Country, COUNT(*) AS num_customers
                   FROM customers
                   GROUP BY Country
                   ORDER BY Country"""

results = pd.read_sql_query(sql_statement, conn)
results

,Country,num_customers
0,Argentina,1
1,Australia,1
2,Austria,1
3,Belgium,1
4,Brazil,5
5,Canada,8
6,Chile,1
7,Czech Republic,2
8,Denmark,1
9,Finland,1


# Question 3
Show how many records are missing from each column in the customers table.

In [ ]:
sql_statement = """SELECT (COUNT(*)-COUNT(CustomerId)) AS num_missing_CustomerId, (COUNT(*)-COUNT(FirstName)) AS num_missing_FirstName, (COUNT(*)-COUNT(LastName)) AS num_missing_LastName, (COUNT(*)-COUNT(Company)) AS num_missing_Company, (COUNT(*)-COUNT(Address)) AS num_missing_Address, (COUNT(*)-COUNT(City)) AS num_missing_City, (COUNT(*)-COUNT(State)) AS num_missing_State, (COUNT(*)-COUNT(Country)) AS num_missing_Country, (COUNT(*)-COUNT(PostalCode)) AS num_missing_PostalCode, (COUNT(*)-COUNT(Phone)) AS num_missing_Phone, (COUNT(*)-COUNT(Fax)) AS num_missing_Fax, (COUNT(*)-COUNT(Email)) AS num_missing_Email, (COUNT(*)-COUNT(SupportRepId)) AS num_missing_SupportRepId
                   FROM customers
                """

results = pd.read_sql_query(sql_statement, conn)
results

,num_missing_CustomerId,num_missing_FirstName,num_missing_LastName,num_missing_Company,num_missing_Address,num_missing_City,num_missing_State,num_missing_Country,num_missing_PostalCode,num_missing_Phone,num_missing_Fax,num_missing_Email,num_missing_SupportRepId
0,0,0,0,49,0,0,29,0,4,1,47,0,0


# Question 4
Find the number of customers in each state of our country

In [ ]:
sql_statement = """SELECT State, COUNT(CustomerId)
                   FROM customers
                   WHERE Country = 'USA'
                   GROUP BY State
             """

results = pd.read_sql_query(sql_statement, conn)
results

,State,COUNT(CustomerId)
0,AZ,1
1,CA,3
2,FL,1
3,IL,1
4,MA,1
5,NV,1
6,NY,1
7,TX,1
8,UT,1
9,WA,1


# Question 5
Select all customers from France who ordered songs in the Opera Genre

In [ ]:
sql_statement = """SELECT DISTINCT(c.CustomerId)
                   FROM customers AS c
                       JOIN invoices AS i
                           ON c.CustomerId = i.CustomerId
                       JOIN invoice_items as i_items
                           ON i.invoiceId = i_items.invoiceId
                       JOIN tracks AS t
                           ON t.TrackId = i_items.TrackId
                       JOIN genres AS g
                           ON g.GenreId = t.GenreId
                   WHERE Country = 'France' AND g.name = 'Metal'"""

results = pd.read_sql_query(sql_statement, conn)
results

,CustomerId
0,39
1,40
2,41
3,42


# Question 6
Find the number of employees under each manager

In [ ]:
sql_statement = """SELECT m.LastName AS manager, COUNT(m.EmployeeId) AS num_employees
                   FROM employees AS e
                       JOIN employees AS m
                       ON e.EmployeeId = m.ReportsTo
                   GROUP BY manager"""

results = pd.read_sql_query(sql_statement, conn)
results

,manager,num_employees
0,Callahan,1
1,Edwards,1
2,Johnson,1
3,King,1
4,Mitchell,1
5,Park,1
6,Peacock,1




# Question 9
How many more times does the USA pay in invoices compared to the U.K.?



In [ ]:
sql_statement = """SELECT SUM(ii.UnitPrice * ii.Quantity)/(SELECT SUM(ii2.UnitPrice * ii2.Quantity)
                                                           FROM customers AS c2
                                                               JOIN invoices AS i2
                                                                  USING (CustomerId)
                                                               JOIN invoice_items AS ii2
                                                                  USING (InvoiceId)
                                                            WHERE c2.Country = 'United Kingdom') AS USA_to_UK_Invoice_Payments
                   FROM customers AS c
                      JOIN invoices AS i
                        USING (CustomerId)
                      JOIN invoice_items AS ii
                        USING (InvoiceId)
                   WHERE Country = 'USA' """

results = pd.read_sql_query(sql_statement, conn)
results

,USA_to_UK_Invoice_Payments
0,4.634592


# Question 10
Show the artists that do not have any albums by using:
One of Union, Union All, Intersect, or Except
An Anti-Join or Semi-Join (so NOT IN or IN)



In [ ]:
sql_statement = """SELECT ArtistId
                   FROM artists

                   EXCEPT

                   SELECT ArtistID
                   FROM albums"""

results = pd.read_sql_query(sql_statement, conn)
results

,ArtistId
0,25
1,26
2,28
3,29
4,30
...,...
66,192
67,193
68,194
69,195


# Question 11
Select all songs from albums that have a longer title than 'Balls to the Walls"

In [ ]:
sql_statement = """SELECT Title, LENGTH(Title) AS num_letters
                   FROM albums
                   WHERE LENGTH(Title) > LENGTH('Balls to the Walls')
                   ORDER BY num_letters DESC"""

results = pd.read_sql_query(sql_statement, conn)
results

,Title,num_letters
0,"Tchaikovsky: 1812 Festival Overture, Op.49, Ca...",95
1,"J.S. Bach: Chaconne, Suite in E Minor, Partita...",86
2,Great Performances - Barber's Adagio and Other...,77
3,"Pure Cult: The Best Of The Cult (For Rockers, ...",76
4,20th Century Masters - The Millennium Collecti...,71
...,...,...
179,Misplaced Childhood,19
180,Every Kind of Light,19
181,Sibelius: Finlandia,19
182,English Renaissance,19



# Question 12
For each track, list the name of the track and the difference in length of the track from the track “Restless and Wild?”



In [ ]:
sql_statement = """SELECT Name, (Milliseconds - (SELECT Milliseconds
                                                 FROM tracks
                                                 WHERE Name = 'Restless and Wild'))
                   FROM tracks"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,(Milliseconds - (SELECT Milliseconds\n FROM tracks\n WHERE Name = 'Restless and Wild'))
0,For Those About To Rock (We Salute You),91668
1,Balls to the Wall,90511
2,Fast As a Shark,-21432
3,Restless and Wild,0
4,Princess of the Dawn,123367
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,34690
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",-112851
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",-185412
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",-30720


# Question 13
Find out the percentage of tracks with bytes bigger than 5  million in the tracks table.

In [ ]:
sql_statement = """SELECT 100.0 * COUNT(Bytes)/(SELECT COUNT(*)
                                        FROM tracks)
                   FROM tracks
                   WHERE Bytes > 5000000"""

results = pd.read_sql_query(sql_statement, conn)
results

,100.0 * COUNT(Bytes)/(SELECT COUNT(*)\n FROM tracks)
0,87.69626


# Question 14
Select albums featuring less than 10 songs.

In [ ]:
sql_statement = """SELECT a.AlbumId, a.Title, COUNT(t.TrackId) AS num_songs
                   FROM albums AS a
                        LEFT JOIN tracks AS t
                        USING (AlbumId)
                   GROUP BY a.Title
                   HAVING COUNT(t.TrackId) < 10
                   ORDER BY num_songs DESC"""

results = pd.read_sql_query(sql_statement, conn)
results

,AlbumId,Title,num_songs
0,156,...And Justice For All,9
1,31,Bongo Fury,9
2,58,Come Taste The Band,9
3,183,Dark Side Of The Moon,9
4,100,Iron Maiden,9
...,...,...,...
132,252,Un-Led-Ed,1
133,275,Vivaldi: The Four Seasons,1
134,287,Wagner: Favourite Overtures,1
135,334,Weill: The Seven Deadly Sins,1


# Question 15
Find the names of the tracks that are MPGE4 File Types


In [ ]:
sql_statement = """SELECT t.Name
                   FROM tracks as t
                        JOIN media_types AS m
                        USING (MediaTypeId)
                   WHERE m.Name LIKE '%MPEG-4%'"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name
0,Battlestar Galactica: The Story So Far
1,Occupation / Precipice
2,"Exodus, Pt. 1"
3,"Exodus, Pt. 2"
4,Collaborators
...,...
209,"There's No Place Like Home, Pt. 2"
210,"There's No Place Like Home, Pt. 3"
211,"Band Members Discuss Tracks from ""Revelations"""
212,Branch Closing


# Question 16
Which tracks, if any, have not been ordered through any invoices?

In [ ]:
sql_statement = """SELECT TrackId
                   FROM tracks

                   EXCEPT

                   SELECT TrackId
                   FROM invoice_items"""

results = pd.read_sql_query(sql_statement, conn)
results

,TrackId
0,7
1,11
2,17
3,18
4,22
...,...
1514,3497
1515,3498
1516,3501
1517,3502


In [ ]:
sql_statement = """SELECT TrackId, Name
                   FROM tracks
                   WHERE TrackId NOT IN (SELECT TrackID
                                         FROM invoice_items)"""

results = pd.read_sql_query(sql_statement, conn)
results

,TrackId,Name
0,7,Let's Get It Up
1,11,C.O.D.
2,17,Let There Be Rock
3,18,Bad Boy Boogie
4,22,Whole Lotta Rosie
...,...,...
1514,3497,"Erlkonig, D.328"
1515,3498,"Concerto for Violin, Strings and Continuo in G..."
1516,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)"
1517,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


# Question 17
Show any city that is the hometown of an employee or customer. Order them in alphabetical order.

In [ ]:
sql_statement = """SELECT City
                   FROM customers

                   UNION

                   SELECT City
                   FROM employees

                   ORDER BY City
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,City
0,Amsterdam
1,Bangalore
2,Berlin
3,Bordeaux
4,Boston
5,Brasília
6,Brussels
7,Budapest
8,Buenos Aires
9,Calgary


# Question 18
Give the name of the track with the largest file size in the tracks table


In [ ]:
sql_statement = """SELECT Name, Bytes
                   FROM tracks
                   WHERE Bytes = (SELECT MAX(Bytes)
                                  FROM tracks)"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,Bytes
0,Through a Looking Glass,1059546140


# Question 20
List any albums with mixed genres


In [ ]:
sql_statement = """SELECT a.AlbumId, a.Title, COUNT(DISTINCT(t.GenreId)) AS num_genres
                   FROM albums AS a
                        JOIN tracks AS t
                        USING (AlbumId)
                   GROUP BY a.Title
                   HAVING num_genres > 1
                   ORDER BY num_genres DESC, AlbumId ASC"""

results = pd.read_sql_query(sql_statement, conn)
results

,AlbumId,Title,num_genres
0,141,Greatest Hits,3
1,227,"Battlestar Galactica, Season 3",3
2,73,Unplugged,2
3,102,Live After Death,2
4,109,Rock In Rio [CD2],2
5,112,The Number of The Beast,2
6,228,"Heroes, Season 1",2
7,229,"Lost, Season 3",2
8,231,"Lost, Season 2",2
9,251,"The Office, Season 3",2


# Question 19
What is the unit price of the track "Fast as a shark?"


In [ ]:
sql_statement = """SELECT t.Name, ii.UnitPrice
                   FROM tracks AS t
                        JOIN invoice_items AS ii
                        USING (TrackId)
                    WHERE t.Name LIKE 'Fast as a Shark'"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,UnitPrice
0,Fast As a Shark,0.99


# Question 21
List the artist against the number of genres they have written. Order artists alphabetically. Show only artists who have written more than one genre.


In [ ]:
sql_statement = """SELECT ar.Name, COUNT(DISTINCT(g.GenreId)) AS num_songs_written
                   FROM genres AS g
                        JOIN tracks AS t
                          USING (GenreId)
                        JOIN albums AS al
                          USING (AlbumId)
                        JOIN artists AS ar
                          USING (ArtistId)
                    GROUP BY ar.Name
                    HAVING num_songs_written > 1
                    ORDER BY ar.Name ASC"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,num_songs_written
0,Amy Winehouse,2
1,Antônio Carlos Jobim,2
2,Audioslave,3
3,Battlestar Galactica,3
4,Eric Clapton,2
5,Faith No More,2
6,Foo Fighters,2
7,Gilberto Gil,3
8,Guns N' Roses,2
9,Heroes,2


# Question 22
Find the 3 artists with the most albums in the database (ordered from most to least albums) Show the id, name and number of albums


In [ ]:
sql_statement = """SELECT ArtistId, Name, COUNT(*) AS num_albums
                   FROM artists
                        LEFT JOIN albums
                        USING (ArtistId)
                   GROUP BY ArtistId
                   ORDER BY num_albums DESC
                   LIMIT 3"""

results = pd.read_sql_query(sql_statement, conn)
results

,ArtistId,Name,num_albums
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11


# Question 23
List the different media types without repeats in the database


In [ ]:
sql_statement = """SELECT DISTINCT(MediaTypeId), Name
                   FROM media_types
                   ORDER BY Name"""

results = pd.read_sql_query(sql_statement, conn)
results

,MediaTypeId,Name
0,5,AAC audio file
1,1,MPEG audio file
2,2,Protected AAC audio file
3,3,Protected MPEG-4 video file
4,4,Purchased AAC audio file


# Question 24
Show how many tracks per playlist, including any playlists that have no tracks.


In [ ]:
sql_statement = """SELECT Name, PlaylistId, COUNT(TrackId)
                   FROM playlists
                        LEFT JOIN playlist_track
                        USING (PlaylistId)
                   GROUP BY PlaylistId
                   ORDER BY name ASC"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,PlaylistId,COUNT(TrackId)
0,90’s Music,5,1477
1,Audiobooks,4,0
2,Audiobooks,6,0
3,Brazilian Music,11,39
4,Classical,12,75
5,Classical 101 - Deep Cuts,13,25
6,Classical 101 - Next Steps,14,25
7,Classical 101 - The Basics,15,25
8,Grunge,16,15
9,Heavy Metal Classic,17,26


# Question 25 *****
Notice there are 2 playlists for tv shows. Count how many songs are common to both lists using: a union, union all, intersect, or except, or b semi join or anti join


In [ ]:
sql_statement = """SELECT COUNT(*) AS num_songs_in_common
                   FROM playlist_track
                   WHERE PlaylistId IN (SELECT PlaylistId
                                        FROM playlists
                                        WHERE Name LIKE '%TV Shows')
                  """

results = pd.read_sql_query(sql_statement, conn)
results

,num_songs_in_common
0,426


# Question 25 alternate attempt
Notice there are 2 playlists for tv shows. Count how many songs are common to both lists using: a union, union all, intersect, or except, or b semi join or anti join


In [ ]:
sql_statement = """SELECT *
                   FROM playlists"""

results = pd.read_sql_query(sql_statement, conn)
results

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music
5,6,Audiobooks
6,7,Movies
7,8,Music
8,9,Music Videos
9,10,TV Shows


# Question 26
Show how many times each song appears in a playlist, showing any songs that have no tracks.


In [ ]:
sql_statement = """SELECT t.Name, COUNT(t.TrackID) AS num_times_in_playlist
                   FROM tracks AS T
                        LEFT JOIN playlist_track AS pt
                        USING (TrackId)
                   GROUP BY t.TrackId
                   ORDER BY num_times_in_playlist DESC"""

results = pd.read_sql_query(sql_statement, conn)
results

,Name,num_times_in_playlist
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
...,...,...
3498,October Song,2
3499,What Is It About Men,2
3500,Help Yourself,2
3501,Amy Amy Amy (Outro),2


# Question 27
Find the min/max/average quantity of items and the min/max/average invoice total for those who live in and out of the U.S.



In [ ]:
sql_statement = """SELECT CASE WHEN c.country = 'USA' THEN 'USA'
                               WHEN c.country != 'USA' THEN 'Other'
                               ELSE 'No Country'
                           END AS usa_or_not,
                           MIN(num_items), MAX(num_items), AVG(num_items), MIN(invoice_total), MAX(invoice_total), AVG(invoice_total)
                   FROM customers AS c
                        JOIN (SELECT i.CustomerId, i.InvoiceId, COUNT(ii.InvoiceLineId) AS num_items, SUM(ii.UnitPrice * ii.Quantity) AS invoice_total
                              FROM invoices AS i
                                    JOIN invoice_items AS ii
                                          USING (InvoiceId)
                              GROUP BY InvoiceId)
                        USING (CustomerId)
                  GROUP BY usa_or_not
                  ORDER BY usa_or_not DESC
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,usa_or_not,MIN(num_items),MAX(num_items),AVG(num_items),MIN(invoice_total),MAX(invoice_total),AVG(invoice_total)
0,USA,1,14,5.428571,0.99,23.86,5.747912
1,Other,1,14,5.439252,0.99,25.86,5.624735
